In [45]:
from numpy.core.numeric import roll
from solid.objects import cylinder, offset, rotate, translate, import_scad, hull
from viewscad import Renderer


r = Renderer()
r.width = 600
r.height = 600

render = r.render


def case(d=60, h=20, tube_size=12, wall=2):
    body = cylinder(d=d, h=h, segments=60)\
        - translate((0, 0, wall))(cylinder(d=d-5, h=h, segments=60))\
        + cylinder(d=4, h=6, segments=60)

    tube_inlet = cylinder(d=tube_size, h=d/2)
    tube_inlet = rotate((90, 0, 0))(tube_inlet)
    tube_offset = d/2-wall-tube_size/2
    tube_inlet_left = translate((-tube_offset, 0, h/2))(tube_inlet)
    tube_inlet_right = translate((tube_offset, 0, h/2))(tube_inlet)
    tube_hole = tube_inlet_left + tube_inlet_right

    return body - tube_hole


def rollers(rolls=3, roll_size=20, d=60, gap=4, wall=2):
    roller_offset = (d-roll_size)/2-wall-gap
    roller = translate((roller_offset, 0, 0)
                       )(cylinder(d=roll_size, h=11, segments=60) - cylinder(d=5, h=12, segments=60))
    rollers = [rotate((0, 0, angle))(roller)
               for angle in range(0, 360, 360//rolls)]

    return sum(rollers)


def base(roll_size=20, d=60, gap=4, wall=2):
    roller_offset = (d-roll_size)/2-wall-gap

    base = cylinder(d=roll_size-gap, h=2)
    base = translate((roller_offset, 0, 0))(base)
    base = hull()(base, cylinder(d=roll_size-gap, h=2))

    return base


def pins(roll_size=20, d=60, gap=4, wall=2):
    roller_offset = (d-roll_size)/2-wall-gap
    pins = translate((roller_offset, 0, 2))(cylinder(d=4, h=12, segments=60))
    pins = pins + translate((roller_offset, 0, 0)
                            )(cylinder(d=3, h=16, segments=60))

    return pins


def roll_mount(rolls=3):
    mount = base() + pins()

    roll_mount = sum([rotate((0, 0, a))(mount)
                     for a in range(0, 360, 360//rolls)])
    roll_mount -= cylinder(d=5, h=2, segments=60)

    return roll_mount


def roll_mount_cap(rolls=3):
    cap = base() - pins()

    cap = sum([rotate((0, 0, a))(cap) for a in range(0, 360, 360//rolls)])
    cap -= cylinder(d=5, h=2, segments=6)
    return cap


scene = roll_mount() + translate((0, 0, 2))(rollers()) + \
    translate((0, 0, 14))(roll_mount_cap())
scene = translate((0, 0, 2))(scene)
scene += case()


render(scene())


render(roll_mount(), outfile='./outfiles/peristaltic_pump_roll_mount.stl')
render(rollers(), outfile='./outfiles/peristaltic_pump_rollers.stl')
render(roll_mount_cap(), outfile='./outfiles/peristaltic_pump_roll_mount_cap.stl')
render(case(), outfile='./outfiles/peristaltic_pump_case.stl')
